In [4]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math

highpass = [[0,-1, 0],
          [-1, 5, -1],
          [0, -1, 0]
         ]
lowpass = [[1/16, 1/8, 1/16],
           [1/8, 1/4, 1/8],
           [1/16, 1/8, 1/16]]

edgeDetection = [[0,-1, 0],
          [-1, 4, -1],
          [0, -1, 0]
         ]

edgeDetection1 = [[-1,-1, -1],
          [-1, 8, -1],
          [-1, -1, -1]
         ]

gaussianBlur = [[1,2, 1],
          [2, 4, 2],
          [1, 2, 1]
         ]

gaussianBlur= np.asarray(gaussianBlur)
gaussianBlur = gaussianBlur/16

In [5]:
def konvolusi(img, kernel):
    kernel = np.asarray(kernel)
    
    centerH = int((kernel.shape[1]-1)/2)
    centerV = int((kernel.shape[0]-1)/2)
    padatas = centerV
    padbawah = kernel.shape[0]-centerV-1
    padkiri = centerH
    padkanan = kernel.shape[1]-centerH-1
    
    #buat lowpass dengan menambahkan pading kiri kanan atas bawah
    lowpass = np.zeros(shape=(img.shape[0]+padatas+padbawah,img.shape[1]+padkiri+padkanan))
    #buat temp yang akan menampung nilai setelah dipangkas
    temp = np.zeros(shape=(img.shape[0],img.shape[1]))
    
    print(temp.shape)
    print(lowpass.shape)
    
    for i in range (img.shape[0]):
        for j in range(img.shape[1]):
            #jika ini pass 1, tambahkan padding ke sisi kanan dan kiri
            lowpass[i+padatas][j+padkiri] = img[i][j]
    
    print(lowpass)
    
    for i in range(temp.shape[0]):
        for j in range(temp.shape[1]):
            for k in range(kernel.shape[0]):
                for l in range(kernel.shape[1]):
                    temp[i][j] += lowpass[i+k][j+l]*kernel[k][l]

    return temp

In [6]:
def limit (img):
    minim = np.min(img)
    maxim = np.max(img)
    
    rang = maxim-minim
    
    img = (img-minim)/rang *255
    img = img.astype(np.uint8).round()
    
    return img

In [7]:
def show (img):
    img = np.asarray(img)
    img = img.astype(np.uint8)
    
    cv2.imshow("Hasil", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [8]:
def changeBrightness(img, b):
    img = img.astype(np.uint16)
    print(img)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if((img[i][j]+b)<256):
                img[i][j]+=b
            else:
                img[i][j] = 255
                
    #img = img.astype(np.uint8)
    return img

In [9]:
def gammaCorrection(img, gamma):
    img = img.astype(np.uint16)
    gamma = 1/gamma
    
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            
            img[i][j] =  math.pow(img[i][j],gamma)
            
    #img = img.astype(np.uint8)
    return img

In [10]:
def meanFilter(img):
    kernel = np.zeros(shape=(3,3))
    centerV = int((kernel.shape[1]-1)/2)
    centerH = int((kernel.shape[0]-1)/2)
    padatas = centerV
    padbawah = kernel.shape[1]-centerV-1
    padkiri = centerH
    padkanan = kernel.shape[0]-centerH-1

    lowpass = np.zeros(shape=(img.shape[0]+padatas+padbawah,img.shape[1]+padkiri+padkanan))
    temp = np.zeros(shape=(img.shape[0],img.shape[1]))
    for i in range (img.shape[0]):
        for j in range(img.shape[1]):
            #jika ini pass 1, tambahkan padding ke sisi kanan dan kiri
            lowpass[i+padatas][j+padkiri] = img[i][j]
    
    #print(lowpass)
    
    for i in range(temp.shape[0]):
        for j in range(temp.shape[1]):
            for k in range(kernel.shape[0]):
                for l in range(kernel.shape[1]):
                    kernel[k][l] = lowpass[i+k][j+l]
    
            temp[i][j] = np.mean(kernel)
    return temp

In [ ]:
citra = cv2.imread('/Users/MUHAMMAD ILHAM/maba.jpg', cv2.IMREAD_GRAYSCALE)

citra1 = np.copy(citra)

print(citra.shape)
#change brigtness
citra = changeBrightness(citra, 20)

citra = np.concatenate((citra, citra1), axis = 1)

show(citra)

#reset citra
citra = np.copy(citra1)

#gamma correction
citra = gammaCorrection(citra, 0.5)
citra = limit(citra)
citra = np.concatenate((citra, citra1), axis = 1)
show(citra) 

#reset citra
citra = np.copy(citra1)

#konvolusi highpass
citra = konvolusi(citra, highpass)
citra = limit(citra)
citra = np.concatenate((citra, citra1), axis = 1)
show(citra)

#reset citra
citra = np.copy(citra1)

#konvolusi lowpass
citra = konvolusi(citra, lowpass)
citra = limit(citra)
citra = np.concatenate((citra, citra1), axis = 1)
show(citra)

#reset citra
citra = np.copy(citra1)

#MeanFilter
citra = meanFilter(citra)
citra = np.round(citra)
citra = np.concatenate((citra, citra1), axis = 1)
show(citra)

(563, 422)
[[ 49  45  46 ...  45  43  42]
 [174 170 169 ... 157 154 153]
 [132 128 128 ... 116 114 114]
 ...
 [ 73  92  75 ...  18  23  36]
 [ 66  68  53 ...  17  22  35]
 [ 51  53  52 ...  16  21  34]]
(563, 422)
(565, 424)
[[  0.   0.   0. ...   0.   0.   0.]
 [  0.  49.  45. ...  43.  42.   0.]
 [  0. 174. 170. ... 154. 153.   0.]
 ...
 [  0.  66.  68. ...  22.  35.   0.]
 [  0.  51.  53. ...  21.  34.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]
(563, 422)
(565, 424)
[[  0.   0.   0. ...   0.   0.   0.]
 [  0.  49.  45. ...  43.  42.   0.]
 [  0. 174. 170. ... 154. 153.   0.]
 ...
 [  0.  66.  68. ...  22.  35.   0.]
 [  0.  51.  53. ...  21.  34.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]
